In [ ]:
#Chain w graph with 108,765 matchings (aka loose adjacency)

In [ ]:
from gerrychain import Graph, Partition, Election
from gerrychain.updaters import Tally, cut_edges
import json
import networkx
from gerrychain import MarkovChain
from gerrychain.constraints import single_flip_contiguous, no_more_discontiguous
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
import pandas
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from functools import partial
from gerrychain.proposals import recom
 
from gerrychain.constraints.validity import within_percent_of_ideal_population
import matplotlib.pyplot as plt
import gerrychain 
import numpy
#NOTE: must have tqdm installed (pip install tqdm in terminal)

In [ ]:
#Use .json of graph 

#NOTE: An island warning will pop up.
#graph = Graph.from_file("/Users/caranix/Documents/MGGG/AK_precincts/alaska_precincts.shp")
#print("graph done")

In [39]:
Ak_graph = Graph.from_file("/Users/caranix/Desktop/Alaska_Git/AK_precincts_ns/AK_precincts_ns.shp")
print("graph done")
#a= Graph.to_json(Ak_graph, "/Users/caranix/Documents/MGGG/AK_graph.json")

graph done


In [ ]:
#Graph.to_json(Ak_graph, "/Users/caranix/Documents/MGGG/AK_graph_ns.json")

In [ ]:
#graph = Graph.from_json("/Users/caranix/Documents/MGGG/AK_graph_ns.json")

In [27]:
def parts_adjacency_matrix(partition):
    parts_graph = networkx.Graph()
    for part in partition.parts:
        parts_graph.add_node(part)
    for edge in partition['cut_edges']:
        source, destination = (partition.assignment[node] for node in edge)
        parts_graph.add_edge(source, destination)
    nodelist = sorted(list(partition.parts.keys()))
    return networkx.to_numpy_matrix(parts_graph, nodelist=nodelist)


In [ ]:
#election = gerrychain.Election("AK", ["PRES16", "USH14", "SEN16","GOV18","USH16","USH18"])

In [28]:
elections = [
    Election("GOV18", {"Democratic": "GOV18D", "Republican": "GOV18R"}),
    Election("USH18", {"Democratic": "USH18D", "Republican": "USH18R"})
]


In [29]:
#updaters = {"population": gerrychain.updaters.Tally("POPULATION", alias="population"), "AK": election, "cut_edges": gerrychain.updaters.cut_edges}
my_updaters = {"population": updaters.Tally("POPULATION", alias="population")}

election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

# Create an initial partition
initial_partition = GeographicPartition(graph, assignment="HDIST", updaters=my_updaters) #NOTE: assignment based on House Districts


In [30]:
#oull adj mat from inital plan 
matrix = []
A= parts_adjacency_matrix(initial_partition) 
matrix.append(A.tolist())
with open('AK_inital_state_2.json', "w") as m:
    m.write(json.dumps(matrix))
print("done")

done


In [40]:
ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)
#print(ideal_population)

proposal = partial(recom,
                   pop_col="POPULATION",
                   pop_target=ideal_population,
                   epsilon=0.05,
                   node_repeats=2
                  )

In [32]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

In [44]:
#THIS CHAIN does work! (thanks Daryl) 

chain = MarkovChain(
    proposal=proposal, 
    constraints=[
        constraints.within_percent_of_ideal_population(initial_partition, .05),
        compactness_bound, no_more_discontiguous
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=10000
)

#NOTE: The current Alaksa plan does not have population within 2%, I increased it to 5% population deviation.

#NOTE: Alaska is not single flip contiguous.
#Changed to no_more_discontiguous, so it doesn't get any worse. 


In [45]:
matrix = []
edge_totals=[]
allAssignments = {0: chain.state.assignment}
for step in chain:
        #allAssignments[chain.counter + 1] = [step.flips]
        allAssignments[chain.counter] = step.assignment
        edge_totals.append((numpy.matrix(parts_adjacency_matrix(step)).sum()/2))
        matrix.append((parts_adjacency_matrix(step).tolist()))        
with open('AK_output10000.json', "w") as m:
    m.write(json.dumps(matrix))
print("Successfully created AK_output10000.json")



KeyError: 397

In [ ]:
chain_dfs = {"GOV18": [],
             "USH18": []}

for key, val in chain_results.items():
    chain_dfs[key] = pandas.DataFrame(val)

In [ ]:
chain_results = {"GOV18": [],
                 "USH18": []}

for partition in chain:
    for key, _ in chain_results.items():
        chain_results[key].append(sorted(partition[key].percents("Democratic"))) 

In [ ]:
print(edge_totals)
print(max(edge_totals))
print(min(edge_totals))


In [ ]:
for i in range(1000):
    with open("./Assignments/AK_House_"+str(i)+".json",'w') as wf:
        json.dump(dict(allAssignments[i]), wf)

In [ ]:
#data = json.dumps(networkx.readwrite.json_graph.adjacency_data(graph))
#It won't let me save graph due to a Polygon error- not a major issue for me atm 
#with open('/Users/caranix/Documents/alaska_graphAK_DATA.json') as f:
#        data = json.load(f)
#graph = networkx.readwrite.json_graph.adjacency_graph(data)
#this is for when I can fix saving the grpah as a .json

In [ ]:
#allAssignments[1]

In [ ]:
#dict(step.assignment)


In [ ]:
chain.counter

In [ ]:
allAssignments

In [ ]:
import geopandas as gpd

In [ ]:
df = gpd.read_file("/Users/caranix/Documents/MGGG/AK_precincts/alaska_precincts.shp")

In [ ]:
for i in range(1000):
    df["plan"+str(i)] = df.index.map(dict(allAssignments[i]))

In [ ]:
plt.figure()
df.plot(column="plan200", cmap = "hsv")
plt.axis('off')


In [ ]:
for i in range(1000):
    if i % 100 == 0:
        plt.figure()
        df.plot(column="plan"+str(i), cmap = "hsv")
        plt.axis('off')
        plt.savefig("./Pictures/AK_plot_"+str(i)+".png", dpi=2000)  
        plt.close()

In [ ]:
new_part = GeographicPartition(graph, assignment=df["plan11"], updaters=my_updaters)#NOTE: assignment based on House Districts


In [ ]:
chain_results = {"GOV18": [],
                 "USH18": []}

for key in chain_results.keys():
    chain_results[key].append(sorted(new_part[key].percents("Democratic"))) 


In [ ]:
pop_vals = [list(partition["population"].values()) for partition in chain]
data= pandas.DataFrame(pop_vals)
print(data)
 

In [ ]:
#house district 1 population values over 1,000 steps of chain 
print(data[0]) 


In [ ]:
#print(data[0].idxmax())
#print(data[0].max())

for i in range(40):
    print("on step "+ str(data[i].idxmax()) + " HD " +str(i+1)+ " reached it's max population: " + str(data[i].max()))

In [ ]:
data.idxmax()
#what step of chain the house district reaches a max. 

In [ ]:
a= data.max(axis=0) #max of each row 
#a.max(axis=0) 
print(a)
a.idxmax()

#max population of a plan 


In [ ]:
data.min(axis=1)

In [ ]:
data.idxmin()

In [ ]:
for i in range(40): 
    data[i].max()

In [ ]:
max_vals = [data[i].max() for i in range(40)]
MAX= max(max_vals)
#max_index= max_vals.maxidx()
#print("max index" +str(max_index))
print(max_vals) #max value for each district 
print("highest population a district has: " +str(MAX)) 

min_vals = [data[i].min() for i in range(40)]
MIN= min(min_vals)
print(min_vals)
print("lowest population a district has: " +str(MIN)) 

print("difference: " + str(MAX-MIN))
#note that this is not within the same plan, but over all of chain 


In [ ]:
#within same plan...

In [ ]:
#deviation btwn single step in chain 

data.max(axis=1)- data.min(axis=1) #max of each row 

In [ ]:
test= data.max(axis=1)- data.min(axis=1)
print(test.idxmax()) #happens on the 426th step of chain 
print(test.max()) #the max divation is 1,754 people between two house districts. 
print(test.max()/ideal_population)

print(test.idxmin()) #happens on the 0th step of chain 
print(test.min()) #the min deviation is 754 people between two house districts. 
print(test.min()/ideal_population)



In [ ]:
plt.figure()
df.plot(column="plan426", cmap = "hsv")
plt.axis('off')


In [ ]:
seat_results= {"GOV18": [],
               "USH18": []}
#print(initial_partition)


#for key, _ in seat_results.items():
 #   seat_results[key].append(initial_partition[key].seats())

In [ ]:
#THIS CHAIN IS  WORKING!! (but it isn't based on proposal, but propose_random_flip)


#chain = gerrychain.MarkovChain(
#    proposal=propose_random_flip,
#    constraints=[constraints.within_percent_of_ideal_population(initial_partition, .03), no_more_discontiguous, compactness_bound],
#    accept=always_accept,
#    initial_state=initial_partition,
#    total_steps=1000
#)
